# i.MXRT118x Signed Flashloader
This is example how to create signed flashloader for i.MXRT118x device with EdgeLock firmware included. The aim of this Jupyter notebook is show how to easily create AHAB container with flashloader application from NXP SDK to be loaded with serial downloader mode.
The process could be split to several steps.

## 1. Prerequisites
- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the [installation](../../_knowledge_base/installation_guide.rst) documentation.)

- This example uses RT1180 EVK board

- Obtain an ELE firmware (mxrt1180b0-ahab-container.img) and put it into the inputs folder. Firmware can be obtained from the SDK package 
  
### 1.1 Compile the flashloader application
As a first step is needed to compile flashloader application itself from NXP SDK. 
  - Download the NXP MCUXpresso SDK from NXP web site [https://mcuxpresso.nxp.com](https://mcuxpresso.nxp.com/)
  - Open the flashloader application in your favorite IDE. (Regular path to application: {SDK_root}\boards\evkmimxrt1180\bootloader_examples\flashloader\cm33)
  - Compile the flashloader and generate the binary output (In our example we use precompiled flashloader_cm33_exe.bin)

### 1.2 Prepare the AHAB configuration file
Run this code to initialize members for this example

In [1]:
import os

from spsdk.utils.jupyter_utils import YamlDiffWidget

# This env variable sets colored logger output to STDOUT
%env JUPYTER_SPSDK=1
# Set a magic for command execution and echo
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


In [2]:
WORKSPACE = "workspace/"  # change this to path to your workspace
VERBOSITY = (
    "-v"  # verbosity of commands, might be -v or -vv for debug or blank for no additional info
)

BIMG_TEMPLATES_PATH = WORKSPACE + "bootable_image_templates"  # Bootable image templates folder
FLSHLDR_EXE_PATH = (
    "inputs/flashloader_cm33_exe.bin"  # Input compiled flashloader executable in binary format
)
AHAB_CONFIG = "inputs/flashloader_cm33_cfg.yaml"  # Prepared AHAB configuration file
AHAB_FILE = (
    WORKSPACE + "flashloader_cm33_ahab.bin"
)  # File name of created signed AHAB container with flashloader and EdgeLockEnclave firmware
BOOTIMG_CONFIG = (
    "inputs/bootimg_rt118x_serial_downloader.yaml"  # Prepared Bootable image configuration file
)
FLASHLOADER = WORKSPACE + "flashloader.bin"  # Final Bootable flashloader

## 2. Prepare AHAB configuration YAML file

Modify the template to configuration file used to create AHAB flashloader as a serial downloader mode


In [3]:
YamlDiffWidget("inputs/rt118x_signed_flashloader.diffc").html

nxpimage ahab get-template -f mimxrt1189 -o workspace/ahab_template.yaml --force 
Creating workspace/ahab_template.yaml template file.


## 3. Export the AHAB container
To export the AHAB container, SPSDK provides a dedicated tool called 'nxpimage' with a specific sub-command group named 'ahab'. This tool is designed to efficiently handle the creation and management of AHAB containers, offering a streamlined process for developers working with NXP's Advanced High-Assurance Boot (AHAB) technology.


In [4]:
# Export AHAB container using nxpimage
%! nxpimage $VERBOSITY ahab export -c $AHAB_CONFIG

assert os.path.exists(AHAB_CONFIG)

nxpimage -v ahab export -c inputs/flashloader_cm33_cfg.yaml 
INFO:spsdk.apps.nxpimage:Created AHAB Image:
Name:      AHAB Image
Starts:    0x0
Ends:      0x177ff
Size:      Size: 94.0 kiB; 96,256 B
Alignment: 8 B
Execution Start Address: Not defined
Pattern:zeros
AHAB Image for mimxrt1189_b0

INFO:spsdk.apps.nxpimage:Created AHAB Image memory map:

+==0x0000_0000= AHAB Image =======================+
|            Size: 94.0 kiB; 96,256 B             |
|          AHAB Image for mimxrt1189_b0           |
|                 Pattern: zeros                  |
|+==0x0000_0000= AHAB Containers ================+|
||            Size: 8.0 kiB; 8,192 B             ||
||             AHAB Containers block             ||
||                Pattern: zeros                 ||
||+==0x0000_0000= AHAB Container 0 =============+||
|||                 Size: 544 B                 |||
|||       AHAB Container for oem_SWver:0        |||
||+==0x0000_021f================================+||
|+==0x0000_1fff==========



## 4. Create a Bootable Image
To properly load the created flashloader over serial downloader mode, a bootable image must be created. In the case of serial downloader, the bootable image is composed of an optional XMCD block (up to 1KB, 0x400B) and the AHAB image itself.

In [5]:
YamlDiffWidget("inputs/rt118x_signed_flashloader_bootable.diffc").html

nxpimage bootable-image get-templates -f mimxrt1189 -o workspace/bootable_image_templates --force 
Creating workspace/bootable_image_templates/bootimg_mimxrt1189_serial_downloader.yaml template file.
Creating workspace/bootable_image_templates/bootimg_mimxrt1189_flexspi_nor.yaml template file.
Creating workspace/bootable_image_templates/bootimg_mimxrt1189_flexspi_nand.yaml template file.
Creating workspace/bootable_image_templates/bootimg_mimxrt1189_semc_nand.yaml template file.
Creating workspace/bootable_image_templates/bootimg_mimxrt1189_emmc.yaml template file.
Creating workspace/bootable_image_templates/bootimg_mimxrt1189_sd.yaml template file.


### 4.1 Export the final bootable image

In [6]:
# Export bootable image

%! nxpimage $VERBOSITY bootable-image merge -c $BOOTIMG_CONFIG -o $FLASHLOADER

assert os.path.exists(FLASHLOADER)

nxpimage -v bootable-image merge -c inputs/bootimg_rt118x_serial_downloader.yaml -o workspace/flashloader.bin 
INFO:spsdk.apps.nxpimage:Created Bootable Image:
Name:      Bootable Image for mimxrt1189
Starts:    0x0
Ends:      0x17bff
Size:      Size: 95.0 kiB; 97,280 B
Alignment: 1 B
Execution Start Address: Not defined
Pattern:zeros
Memory type: MemoryType.SERIAL_DOWNLOADER
Revision: latest

INFO:spsdk.apps.nxpimage:Created Bootable Image memory map:

+==0x0000_0000= Bootable Image for mimxrt1189 ==+
|           Size: 95.0 kiB; 97,280 B            |
|   Memory type: MemoryType.SERIAL_DOWNLOADER   |
|               Revision: latest                |
|                Pattern: zeros                 |
|                 Gap: 1.0 kiB                  |
|+==0x0000_0400= ahab_container ===============+|
||          Size: 94.0 kiB; 96,256 B           ||
|+==0x0001_7bff================================+|
+==0x0001_7bff==================================+

Success. (Bootable Image: workspace/flash

## 5. Generated flashloader file load to i.MXRT118x
As the final step, load the flashloader into the MCU and run it. The easiest way to do this is by using the 'blhost' application with the 'load-image' command.

Ensure that the chip is in serial downloader mode!

Here are two examples of how to do this over a Serial line or USB:
- `blhost -p com1 load-image workspace/flashloader.bin`
- `blhost -u 0x1fc9:0x014c load-image workspace/flashloader.bin`